# Lab Activity: Working with Mistrtal
## Noor Elburdini (60300690)

In [1]:
!pip install mistralai


  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.1
    Uninstalling PyYAML-6.0.1:
      Successfully uninstalled PyYAML-6.0.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0


In [2]:
!pip install -r requirements.txt


Load API key and helper function

In [3]:
import os
os.environ["MISTRAL_API_KEY"] = "fr006GjkNB8tbu5PQ7ODgvT9JFYg7hYn"
print(f"MISTRAL_API_KEY: {os.environ.get('MISTRAL_API_KEY')}")


MISTRAL_API_KEY: fr006GjkNB8tbu5PQ7ODgvT9JFYg7hYn


In [4]:
api_key = os.getenv("MISTRAL_API_KEY")


In [5]:
from mistralai import Mistral, UserMessage

def mistral(user_message, model="mistral-small-latest", is_json=False):
    model = "mistral-large-latest"
    client = Mistral(api_key=api_key)

    messages = [UserMessage(content=user_message)]

    chat_response = client.chat.complete(
        model=model,
        messages=messages,
    )
    return chat_response.choices[0].message.content


In [6]:
mistral("hello, what can you do?")


'Hello! I’m an AI assistant here to help with a wide range of tasks. Here’s what I can do for you:\n\n### **General Assistance**\n- Answer questions (facts, explanations, advice).\n- Summarize articles, books, or documents.\n- Help with brainstorming ideas (writing, projects, business, etc.).\n- Provide step-by-step guides (recipes, DIY, tech setup, etc.).\n\n### **Writing & Editing**\n- Write or edit emails, essays, reports, or creative stories.\n- Generate blog posts, social media captions, or marketing copy.\n- Improve grammar, clarity, and style in your writing.\n- Translate text between languages (though not perfectly).\n\n### **Learning & Research**\n- Explain complex topics (science, history, philosophy, etc.).\n- Help with homework or study guides (math, coding, languages, etc.).\n- Find and summarize research papers or news articles.\n- Recommend books, courses, or learning resources.\n\n### **Productivity & Organization**\n- Create to-do lists, schedules, or planners.\n- Help

## Classification

In [7]:
prompt = """
You are a bank customer service bot.
Your task is to assess customer intent and categorize customer
inquiry after <<<>>> into one of the following predefined categories:

card arrival
change pin
exchange rate
country support
cancel transfer
charge dispute

If the text doesn't fit into any of the above categories,
classify it as:

customer service

You will only respond with the predefined category.
Do not provide explanations or notes.

###
Here are some examples:

Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card.
Category: card arrival

Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees.
Category: exchange rate

Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time.
Category: country support

Inquiry: Can I get help starting my computer? I am having difficulty starting my computer.
Category: customer service

###
<<<
Inquiry: {inquiry}
>>>
Category:
"""


### Ask Mistral to check the spelling and grammar of your prompt

In [8]:
response = mistral(f"Please correct the spelling and grammar of \
this prompt and return a text that is the same prompt,\
with the spelling and grammar fixed: {prompt}")

In [9]:
print(response)

Here’s your corrected prompt with proper spelling and grammar:

---

You are a bank customer service bot.
Your task is to assess customer intent and categorize the customer inquiry within the `<<<>>>` markers into one of the following predefined categories:

- card arrival
- change pin
- exchange rate
- country support
- cancel transfer
- charge dispute

If the text does not fit into any of the above categories, classify it as:

- customer service

You will respond **only** with the predefined category.
Do not provide explanations or notes.

###
Here are some examples:

**Inquiry:** How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card.
**Category:** card arrival

**Inquiry:** I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees.
**Category:** exchange rate

**Inquiry:** What countries are supported? 

Try out the model

In [10]:
mistral(
    prompt.format(
        inquiry="I am inquiring about the availability of your cards in the EU"
    )
)


'country support'

## Information Extraction with JSON Mode

In [11]:
medical_notes = """
A 60-year-old male patient, Mr. Johnson, presented with symptoms
of increased thirst, frequent urination, fatigue, and unexplained
weight loss. Upon evaluation, he was diagnosed with diabetes,
confirmed by elevated blood sugar levels. Mr. Johnson's weight
is 210 lbs. He has been prescribed Metformin to be taken twice daily
with meals. It was noted during the consultation that the patient is
a current smoker.
"""

In [12]:
prompt = f"""
Extract information from the following medical notes:
{medical_notes}

Return json format with the following JSON schema:
{{
  "age": {{
    "type": "integer"
  }},
  "gender": {{
    "type": "string",
    "enum": ["male", "female", "other"]
  }},
  "diagnosis": {{
    "type": "string",
    "enum": ["migraine", "diabetes", "arthritis", "acne"]
  }},
  "weight": {{
    "type": "integer"
  }},
  "smoking": {{
    "type": "string",
    "enum": ["yes", "no"]
  }}
}}
"""


In [13]:
response = mistral(prompt, is_json=True)
print(response)


```json
{
  "age": 60,
  "gender": "male",
  "diagnosis": "diabetes",
  "weight": 210,
  "smoking": "yes"
}
```


## Personalization

In [14]:
email = """

Dear mortgage lender,

What's your 30-year fixed-rate APR, how is it compared to the 15-year
fixed rate?

Regards,
Anna
"""

In [15]:
prompt = f"""
You are a mortgage lender customer service bot, and your task is to
create personalized email responses to address customer questions.
Answer the customer's inquiry using the provided facts below. Ensure
that your response is clear, concise, and directly addresses the
customer's question. Address the customer in a friendly and
professional manner. Sign the email with
"Lender Customer Support."
# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%
# Email
{email} """

In [16]:
response = mistral(prompt, is_json=True)
print(response)


**Subject:** Your 30-Year and 15-Year Fixed-Rate APR Details

Dear Anna,

Thank you for reaching out! Here are the details you requested:

- **30-year fixed-rate:** The APR is **6.484%**.
- **15-year fixed-rate:** The APR is **5.848%**.

The 15-year fixed-rate offers a lower APR compared to the 30-year option, which typically means lower overall interest costs over the life of the loan. However, the shorter term also comes with higher monthly payments.

Would you like help comparing these options further or exploring other loan types? Let me know how I can assist!

Best regards,
**Lender Customer Support**


## Summarization

In [17]:
newsletter = """
European AI champion Mistral AI unveiled new large language models and formed an alliance with Microsoft.

What’s new: Mistral AI introduced two closed models, Mistral Large and Mistral Small (joining Mistral Medium, which debuted quietly late last year).

Model specs: The new models’ parameter counts, architectures, and training methods are undisclosed.

Mistral Large achieved 81.2 percent on the MMLU benchmark, outperforming Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B.

Both models are fluent in French, German, Spanish, and Italian. They’re trained for function calling and JSON-format output.

Microsoft’s investment in Mistral AI is significant but tiny compared to its investments in other AI companies.

Mistral AI and Microsoft will collaborate to train bespoke models for customers including European governments.

Behind the news: Mistral AI was founded in early 2023 by engineers from Google and Meta.

Yes, but: Mistral AI’s partnership with Microsoft has divided European lawmakers and regulators.

Why it matters: The partnership between Mistral AI and Microsoft gives the startup crucial processing power for training large models and greater access to potential customers.
We’re thinking: Mistral AI has made impressive progress in a short time, especially relative to the resources at its disposal as a startup. Its partnership with Microsoft could significantly accelerate its ability to compete with larger AI players, though it may also intensify regulatory and competitive tensions within the European AI ecosystem.
"""


In [18]:
response = mistral(prompt)
print(response)


**Subject:** Your 30-Year and 15-Year Fixed-Rate APR Details

Dear Anna,

Thank you for reaching out! Here are the details you requested:

- **30-year fixed-rate:** The APR is **6.484%**.
- **15-year fixed-rate:** The APR is **5.848%**.

The 15-year fixed-rate offers a lower APR compared to the 30-year option, which typically means lower overall interest costs over the life of the loan. However, the shorter term also comes with higher monthly payments.

Would you like help comparing these options further or exploring other loan types? Let me know how I can assist!

Best regards,
**Lender Customer Support**


In [19]:
prompt = f"""
You are a commentator. Your task is to write a report on a newsletter.
When presented with the newsletter, come up with interesting questions to ask,
and answer each question.
Afterward, combine all the information and write a report in the markdown
format.
# Newsletter:
{newsletter}
# Instructions:
## Summarize:
In clear and concise language, summarize the key points and themes
presented in the newsletter.
## Interesting Questions:
Generate three distinct and thought-provoking questions that can be
asked about the content of the newsletter. For each question:
-
After
"Q:
"
, describe the problem
-
After
"A:
"
, provide a detailed explanation of the problem addressed
in the question.
-
Enclose the ultimate answer in <>.
## Write a analysis report
Using the summary and the answers to the interesting questions,
create a comprehensive report in Markdown format.
"""

In [20]:
response = mistral(prompt)
print(response)


# **Analysis Report: Mistral AI’s New Models and Microsoft Partnership**

## **Summary**
Mistral AI, a European AI startup founded in 2023 by former Google and Meta engineers, has unveiled two new closed large language models (LLMs): **Mistral Large** and **Mistral Small**, joining the previously released **Mistral Medium**. Key highlights include:
- **Performance**: Mistral Large outperforms competitors like Claude 2, Gemini Pro, and Llama 2 70B on the MMLU benchmark (81.2%).
- **Multilingual & Functional Capabilities**: Fluent in French, German, Spanish, and Italian, with support for function calling and JSON output.
- **Microsoft Partnership**: A strategic alliance to train custom models for European governments, despite Microsoft’s relatively small investment compared to its other AI ventures.
- **Regulatory & Competitive Tensions**: The partnership has sparked debate among European lawmakers and regulators, raising concerns about market dominance and data sovereignty.

The collabo